In [2]:
!python -V

Python 3.9.19


In [3]:
import pandas as pd

In [4]:
import pickle

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [7]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-Experiment-tracking/mlruns/1', creation_time=1726570807518, experiment_id='1', last_update_time=1726570807518, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [8]:
def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('./data/green_tripdata_2023-01.csv')
df_val = read_dataframe('./data/green_tripdata_2023-02.csv')

/tmp/ipykernel_2768/3970424726.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [10]:
len(df_train), len(df_val)

(65946, 62574)

In [11]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [12]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [13]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [14]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


np.float64(6.037803764245777)

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [23]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2023-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2023-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [17]:
import xgboost as xgb

In [18]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [20]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:35:46] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.61299                           
[1]	validation-rmse:8.01900                           
[2]	validation-rmse:7.52461                           
[3]	validation-rmse:7.11607                           
[4]	validation-rmse:6.78018                           
[5]	validation-rmse:6.50608                           
[6]	validation-rmse:6.28445                           
[7]	validation-rmse:6.10537                           
[8]	validation-rmse:5.96202                           
[9]	validation-rmse:5.84661                           
[10]	validation-rmse:5.75391                          
[11]	validation-rmse:5.67745                          
[12]	validation-rmse:5.61782                          
[13]	validation-rmse:5.56949                          
[14]	validation-rmse:5.52962                          
[15]	validation-rmse:5.49889                          
[16]	validation-rmse:5.47407                          
[17]	validation-rmse:5.45360                          
[18]	valid

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:37:50] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.25517                                                        
[1]	validation-rmse:7.43436                                                        
[2]	validation-rmse:6.81593                                                        
[3]	validation-rmse:6.35783                                                        
[4]	validation-rmse:6.02007                                                        
[5]	validation-rmse:5.77782                                                        
[6]	validation-rmse:5.60235                                                        
[7]	validation-rmse:5.47707                                                        
[8]	validation-rmse:5.38814                                                        
[9]	validation-rmse:5.32723                                                        
[10]	validation-rmse:5.28184                                                       
[11]	validation-rmse:5.25049                                                

...

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [12:26:47] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.62016                                                     
[1]	validation-rmse:8.02645                                                     
[2]	validation-rmse:7.52760                                                     
[3]	validation-rmse:7.11007                                                     
[4]	validation-rmse:6.76464                                                     
[5]	validation-rmse:6.47974                                                     
[6]	validation-rmse:6.24565                                                     
[7]	validation-rmse:6.05600                                                     
[8]	validation-rmse:5.90170                                                     
[9]	validation-rmse:5.77533                                                     
[10]	validation-rmse:5.67318                                                    
[11]	validation-rmse:5.59177                                                    
[12]	validation-rmse:5.52449

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [12:28:12] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.91303                                                     
[1]	validation-rmse:6.95978                                                     
[2]	validation-rmse:6.32727                                                     
[3]	validation-rmse:5.92380                                                     
[4]	validation-rmse:5.66991                                                     
[5]	validation-rmse:5.50473                                                     
[6]	validation-rmse:5.40704                                                     
[7]	validation-rmse:5.33940                                                     
[8]	validation-rmse:5.29346                                                     
[9]	validation-rmse:5.26351                                                     
[10]	validation-rmse:5.24350                                                    
[11]	validation-rmse:5.23107                                                    
[12]	validation-rmse:5.21479

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [21]:
mlflow.xgboost.autolog(disable=True)

In [22]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.06273447100027489,
        'max_depth': 52,
        'min_child_weight': 3.372889853990348,
        'objective': 'reg:linear',
        'reg_alpha': 0.06919062960886975,
        'reg_lambda': 0.024791137697948,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [10:49:33] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.92920
[1]	validation-rmse:8.56914
[2]	validation-rmse:8.23995
[3]	validation-rmse:7.93935
[4]	validation-rmse:7.66533
[5]	validation-rmse:7.41617
[6]	validation-rmse:7.18995
[7]	validation-rmse:6.98521
[8]	validation-rmse:6.79965
[9]	validation-rmse:6.63266
[10]	validation-rmse:6.48169
[11]	validation-rmse:6.34575
[12]	validation-rmse:6.22383
[13]	validation-rmse:6.11404
[14]	validation-rmse:6.01495
[15]	validation-rmse:5.92690
[16]	validation-rmse:5.84817
[17]	validation-rmse:5.77689
[18]	validation-rmse:5.71396
[19]	validation-rmse:5.65699
[20]	validation-rmse:5.60614
[21]	validation-rmse:5.56119
[22]	validation-rmse:5.52026
[23]	validation-rmse:5.48388
[24]	validation-rmse:5.45155
[25]	validation-rmse:5.42243
[26]	validation-rmse:5.39671
[27]	validation-rmse:5.37452
[28]	validation-rmse:5.35457
[29]	validation-rmse:5.33566
[30]	validation-rmse:5.31984
[31]	validation-rmse:5.30507
[32]	validation-rmse:5.29177
[33]	validation-rmse:5.28022
[34]	validation-rmse:5.2

/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/new-exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [10:51:43] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2024/09/19 10:51:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [25]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2023-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2023-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

2024/09/19 11:14:30 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.1, but the installed version is 1.5.2. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2024/09/19 11:14:30 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


KeyboardInterrupt: 

In [26]:

logged_model = 'runs:/920b8deac31a43cba7d748b5332733f7/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


In [28]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 920b8deac31a43cba7d748b5332733f7